# About: Hadoop Prerequisites - Ready! on CentOS6

---

Hadoopをマシンに配備するため、Prerequisite Playbookを適用する。なお、PlaybookはNIIクラウド担当のプライベートなGitLabにて管理している。

## *Operation Note*

*This is a cell for your own recording.  ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。

In [1]:
target_group = 'test-hadoop-vm'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.70 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.71 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.113 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.73 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.112 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.114 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


/etc/hostsの生成に必要なため、ホスト表ファイルの読み込みも行います。

In [3]:
# ホスト表ファイルのパス
hosts_csv = 'hosts.csv'

# 対象クラスタ名
target_cluster = 'TestCluster'

In [4]:
%run scripts/loader.py

header, machines = read_machines(hosts_csv)
machines = filter(lambda m: m['Cluster'] == target_cluster, machines)
pd.DataFrame(map(lambda m: get_row(header, m), machines), columns=header)

,Cluster,Type,Name,Internal IP,Service IP,VCPUs,Memory(MiB),DFS Volumes,YARN VCPUs,YARN Total Memory(MB),...,HBase RegionServer,Tez,Hive,Pig,Client,Spark,Spark HistoryServer,KDC Master,KDC Slave,Docker
0,TestCluster,vm,testvm001,,XXX.XXX.XXX.70,2,10240,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,TestCluster,vm,testvm002,,XXX.XXX.XXX.71,2,10240,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,TestCluster,vm,testvm003,,XXX.XXX.XXX.72,2,10240,NaN,NaN,NaN,...,False,True,True,True,True,True,True,False,False,False
3,TestCluster,vm,testvm004,,XXX.XXX.XXX.73,2,10240,1.0,1.0,4096.0,...,True,False,False,False,False,False,False,False,False,False
4,TestCluster,vm,testvm005,,XXX.XXX.XXX.112,2,10240,1.0,1.0,4096.0,...,True,False,False,False,False,False,False,False,False,False
5,TestCluster,vm,testvm006,,XXX.XXX.XXX.113,2,10240,1.0,1.0,4096.0,...,True,False,False,False,False,False,False,False,False,False
6,TestCluster,vm,testvm007,,XXX.XXX.XXX.114,2,10240,1.0,1.0,4096.0,...,True,False,False,False,False,False,False,False,False,False


# Prerequisite Playbookの適用

Prerequisite Playbookのダウンロードと適用をおこなう。**Prerequisite Playbookはベースとなる環境により異なります。この情報はNIIのクラウド環境固有のものであるため、非公開とします。**

HDPを動作させるためには、以下の操作をおこなう必要があります。

- IPv6の無効化
- NTP設定を適切なものに変更
- パッケージの更新
- SELinuxの無効化

In [5]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmpEVmuhr'

Playbookをcloneする。**(NIIクラウドのプライベートなリポジトリを使っています。非公開)**

In [6]:
!git clone ssh://xxx.nii.ac.jp/xxx/aic-dataanalysis-prerequisite.git {work_dir}/playbook

Cloning into '/tmp/tmpEVmuhr/playbook'...
remote: Counting objects: 399, done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 399 (delta 138), reused 196 (delta 55)
Receiving objects: 100% (399/399), 37.95 KiB | 0 bytes/s, done.
Resolving deltas: 100% (138/138), done.
Checking connectivity... done.


cloneされたファイルの構成を、念のため確認しておく。

In [7]:
!tree {work_dir}/playbook

/tmp/tmpEVmuhr/playbook
├── ansible.cfg
├── documents
├── group_vars
│   ├── cn0107-centos6
│   ├── cn0107-centos7
│   ├── sn0202
│   └── sn0203
├── hosts-aic
├── mount.yml
├── playbooks
│   ├── conf
│   │   ├── all-aic.yml
│   │   ├── common.yml
│   │   ├── network.yml
│   │   ├── ntp.yml
│   │   ├── packages.yml
│   │   └── ssh.yml
│   └── destructive
│       └── clean-hdds.yml
└── roles
    ├── common
    │   ├── defaults
    │   │   └── main.yml
    │   ├── tasks
    │   │   ├── ipv6.yml
    │   │   └── main.yml
    │   └── templates
    │       └── disable_ipv6.conf.j2
    ├── coreutils
    │   └── tasks
    │       ├── main.yml
    │       └── update.yml
    ├── mount
    │   └── tasks
    │       └── main.yml
    ├── network
    │   ├── handlers
    │   │   └── main.yml
    │   ├── tasks
    │   │   └── main.yml
    │   └── templates
    │       ├── etc_hosts.j2
    │       ├── etc_resolv.conf.j2
    │       ├── ifcfg-bond0-CN.j2
    │       ├── ifcfg-eth1-sn0202.j2
    │       

今回はVMでのHadoop実行をおこなうため、以下のroleを適用することにする。

- common ... IPv6の無効化
- ntp ... NTP設定を所内向け設定に変更
- packages ... パッケージの更新
- selinux ... SELinuxの無効化

In [8]:
with open('{work_dir}/playbook/site.yml'.format(work_dir=work_dir), 'w') as f:
    f.write('''- hosts: {target_group}
  become: yes
  roles:
    - common
    - ntp
    - packages
    - selinux'''.format(target_group=target_group))
!cat {work_dir}/playbook/site.yml

- hosts: test-hadoop-vm
  become: yes
  roles:
    - common
    - ntp
    - packages
    - selinux

Ansible Playbookを適用する・・・

In [9]:
!ansible-playbook -CDv {work_dir}/playbook/site.yml

Using /etc/ansible/ansible.cfg as config file

PLAY [test-hadoop-vm] **********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.72]
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.114]

TASK [common : include] ********************************************************
[DEPRECATION WARNING]: You should not specify tags in the include parameters. 
All tags should be specified using the task-level option.
This feature will be 
removed in a future release. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.
included: /tmp/tmpEVmuhr/playbook/roles/common/tasks/ipv6.yml for XXX.XXX.XXX.70, XXX.XXX.XXX.71, XXX.XXX.XXX.72, XXX.XXX.XXX.73, XXX.XXX.XXX.112, XXX.XXX.XXX.113, XXX.XXX.XXX.114

TASK [common : disable ipv6] ***************************************************
changed: [XXX.XX

In [10]:
!ansible-playbook {work_dir}/playbook/site.yml


PLAY [test-hadoop-vm] **********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.70]
ok: [XXX.XXX.XXX.71]
ok: [XXX.XXX.XXX.72]
ok: [XXX.XXX.XXX.73]
ok: [XXX.XXX.XXX.112]
ok: [XXX.XXX.XXX.113]
ok: [XXX.XXX.XXX.114]

TASK [common : include] ********************************************************
[DEPRECATION WARNING]: You should not specify tags in the include parameters. 
All tags should be specified using the task-level option.
This feature will be 
removed in a future release. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.
included: /tmp/tmpEVmuhr/playbook/roles/common/tasks/ipv6.yml for XXX.XXX.XXX.70, XXX.XXX.XXX.71, XXX.XXX.XXX.72, XXX.XXX.XXX.73, XXX.XXX.XXX.112, XXX.XXX.XXX.113, XXX.XXX.XXX.114

TASK [common : disable ipv6] ***************************************************
changed: [XXX.XXX.XXX.71] => (item=disable_ipv6.conf)
changed:

## 時刻同期の確認

NTPの同期状態を確認する `ntpXX.sinet.ad.jp` に `*`, `+` のマークがついていればOK。

In [15]:
!ansible -b -a 'ntpq -p' {target_group}

XXX.XXX.XXX.71 | SUCCESS | rc=0 >>
     remote           refid      st t when poll reach   delay   offset  jitter
*ntp01.sinet.ad. .GPS.            1 u    6   64    3    1.958  -29.461   8.767
+ntp11.sinet.ad. .GPS.            1 u    6   64    3    1.209  -29.599   8.919

XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
     remote           refid      st t when poll reach   delay   offset  jitter
*ntp01.sinet.ad. .GPS.            1 u    9   64    3    1.780   -0.342   0.038
+ntp11.sinet.ad. .GPS.            1 u    7   64    3    1.104   -0.426   0.051

XXX.XXX.XXX.72 | SUCCESS | rc=0 >>
     remote           refid      st t when poll reach   delay   offset  jitter
*ntp01.sinet.ad. .GPS.            1 u    9   64    3    1.869   -0.367   0.200
+ntp11.sinet.ad. .GPS.            1 u    5   64    3    1.246   -0.343   0.210

XXX.XXX.XXX.73 | SUCCESS | rc=0 >>
     remote           refid      st t when poll reach   delay   offset  jitter
*ntp01.sinet.ad. .GPS.            1 u    7   64    3    1.922   -0.

## その他必要なツールのインストール

JDKをインストールする際に、wgetが必要。

In [16]:
!ansible -b -m yum -a 'name=wget' {target_group}

XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "msg": "", 
    "rc": 0, 
    "results": [
        "Loaded plugins: fastestmirror\nSetting up Install Process\nLoading mirror speeds from cached hostfile\n * base: ftp.iij.ad.jp\n * extras: ftp.iij.ad.jp\n * updates: ftp.iij.ad.jp\nResolving Dependencies\n--> Running transaction check\n---> Package wget.x86_64 0:1.12-8.el6 will be installed\n--> Finished Dependency Resolution\n\nDependencies Resolved\n\n================================================================================\n Package         Arch              Version                Repository       Size\n================================================================================\nInstalling:\n wget            x86_64            1.12-8.el6             base            484 k\n\nTransaction Summary\n================================================================================\nInstall       1 Package(s)\n\nTotal download size: 484 k\nInstalled size: 1.8 M\nDownloading 

## hostnameの設定

マシンにホスト名を設定する。

In [17]:
for m in machines:
    !ansible -b -m hostname -a "name={m['Name']}" {m['Service IP']}

XXX.XXX.XXX.70 | SUCCESS => {
    "ansible_facts": {
        "ansible_domain": "", 
        "ansible_fqdn": "testvm001", 
        "ansible_hostname": "testvm001", 
        "ansible_nodename": "testvm001"
    }, 
    "changed": true, 
    "name": "testvm001"
}
XXX.XXX.XXX.71 | SUCCESS => {
    "ansible_facts": {
        "ansible_domain": "", 
        "ansible_fqdn": "testvm002", 
        "ansible_hostname": "testvm002", 
        "ansible_nodename": "testvm002"
    }, 
    "changed": true, 
    "name": "testvm002"
}
XXX.XXX.XXX.72 | SUCCESS => {
    "ansible_facts": {
        "ansible_domain": "", 
        "ansible_fqdn": "testvm003", 
        "ansible_hostname": "testvm003", 
        "ansible_nodename": "testvm003"
    }, 
    "changed": true, 
    "name": "testvm003"
}
XXX.XXX.XXX.73 | SUCCESS => {
    "ansible_facts": {
        "ansible_domain": "", 
        "ansible_fqdn": "testvm004", 
        "ansible_hostname": "testvm004", 
        "ansible_nodename": "testvm004"
    }, 
    "cha

## /etc/hostsの変更

hosts.csvに基づき、/etc/hostsを変更する。

In [18]:
with open('{work_dir}/hosts'.format(work_dir=work_dir), 'w') as f:
    f.write('''127.0.0.1   localhost localhost.localdomain localhost4 localhost4.localdomain4

{all_hosts}
'''.format(all_hosts='\n'.join(map(lambda m: '{} {}'.format(m['Service IP'], m['Name']), machines))))

!cat {work_dir}/hosts

XXX.XXX.XXX.1   localhost localhost.localdomain localhost4 localhost4.localdomain4

XXX.XXX.XXX.70 testvm001
XXX.XXX.XXX.71 testvm002
XXX.XXX.XXX.72 testvm003
XXX.XXX.XXX.73 testvm004
XXX.XXX.XXX.112 testvm005
XXX.XXX.XXX.113 testvm006
XXX.XXX.XXX.114 testvm007


変更内容を確認するため、まずはdry-runする。

In [19]:
!ansible -CDv -b -m copy -a 'src={work_dir}/hosts dest=/etc/hosts' {target_group}

Using /etc/ansible/ansible.cfg as config file
XXX.XXX.XXX.70 | SUCCESS => {
    "changed": true, 
    "dest": "/etc/hosts", 
    "src": "/tmp/tmpEVmuhr/hosts"
}
--- before: /etc/hosts
+++ after: /tmp/tmpEVmuhr/hosts
@@ -1,2 +1,9 @@
 XXX.XXX.XXX.1   localhost localhost.localdomain localhost4 localhost4.localdomain4
-::1         localhost localhost.localdomain localhost6 localhost6.localdomain6
+
+XXX.XXX.XXX.70 testvm001
+XXX.XXX.XXX.71 testvm002
+XXX.XXX.XXX.72 testvm003
+XXX.XXX.XXX.73 testvm004
+XXX.XXX.XXX.112 testvm005
+XXX.XXX.XXX.113 testvm006
+XXX.XXX.XXX.114 testvm007

XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "dest": "/etc/hosts", 
    "src": "/tmp/tmpEVmuhr/hosts"
}
--- before: /etc/hosts
+++ after: /tmp/tmpEVmuhr/hosts
@@ -1,2 +1,9 @@
 XXX.XXX.XXX.1   localhost localhost.localdomain localhost4 localhost4.localdomain4
-::1         localhost localhost.localdomain localhost6 localhost6.localdomain6
+
+XXX.XXX.XXX.70 testvm001
+XXX.XXX.XXX.71 testvm002
+XXX.XXX.XXX

差分を確認し、意図と異なる変更がないことを確認したうえで、変更処理を実施する。

In [20]:
!ansible -b -m copy -a 'src={work_dir}/hosts dest=/etc/hosts' {target_group}

XXX.XXX.XXX.70 | SUCCESS => {
    "changed": true, 
    "checksum": "ada94b4db6b180f89b03d966e5667530cd351442", 
    "dest": "/etc/hosts", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "437a619f84bff68fc53ef39f84e42612", 
    "mode": "0644", 
    "owner": "root", 
    "size": 244, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1472179406.0-39231455556314/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.71 | SUCCESS => {
    "changed": true, 
    "checksum": "ada94b4db6b180f89b03d966e5667530cd351442", 
    "dest": "/etc/hosts", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "437a619f84bff68fc53ef39f84e42612", 
    "mode": "0644", 
    "owner": "root", 
    "size": 244, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1472179406.04-188815258677144/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": true, 
    "checksum": "ada94b4db6b180f89b03d966e5667530cd351442", 
    "dest": "/etc/hosts", 
    "gid": 0, 
    "group": 

各ホスト間で意図した通り名前解決できることを確認する。

全組み合わせ試すのは面倒なので、MasterとなるNodeから、SlaveとなるNodeへの疎通を確認する。

In [21]:
for src in filter(lambda m: m['NameNode'], machines):
    for dest in filter(lambda m: m['DataNode'], machines):
        print('{} -> {}'.format(src['Name'], dest['Name']))
        !ansible -a "ping -c 4 {dest['Name']}" {src['Service IP']}

testvm001 -> testvm004
XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
PING testvm004 (XXX.XXX.XXX.73) 56(84) bytes of data.
64 bytes from testvm004 (XXX.XXX.XXX.73): icmp_seq=1 ttl=64 time=1.66 ms
64 bytes from testvm004 (XXX.XXX.XXX.73): icmp_seq=2 ttl=64 time=0.284 ms
64 bytes from testvm004 (XXX.XXX.XXX.73): icmp_seq=3 ttl=64 time=0.279 ms
64 bytes from testvm004 (XXX.XXX.XXX.73): icmp_seq=4 ttl=64 time=0.217 ms

--- testvm004 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3002ms
rtt min/avg/max/mdev = 0.217/0.611/1.664/0.608 ms

testvm001 -> testvm005
XXX.XXX.XXX.70 | SUCCESS | rc=0 >>
PING testvm005 (XXX.XXX.XXX.112) 56(84) bytes of data.
64 bytes from testvm005 (XXX.XXX.XXX.112): icmp_seq=1 ttl=64 time=0.843 ms
64 bytes from testvm005 (XXX.XXX.XXX.112): icmp_seq=2 ttl=64 time=0.131 ms
64 bytes from testvm005 (XXX.XXX.XXX.112): icmp_seq=3 ttl=64 time=0.112 ms
64 bytes from testvm005 (XXX.XXX.XXX.112): icmp_seq=4 ttl=64 time=0.125 ms

--- testvm005 ping statistics 

# 後始末

一時ディレクトリを削除する。

In [22]:
!rm -fr {work_dir}